In [5]:
!pip install llmware


In [8]:
import os
import re
from llmware.prompts import Prompt, HumanInTheLoop
from llmware.configs import LLMWareConfig
from llmware.setup import Setup
from llmware.models import ModelCatalog

class InvoiceProcessor:
    def __init__(self):
        self.prompter = None

    def initialize_prompter(self, run_on_cpu=True):
        print("Initializing...")
        try:
            sample_files_path = Setup().load_sample_files(over_write=False)
            invoices_path = os.path.join(sample_files_path, "Invoices")

            query_list = [
                "What is the total amount of the invoice?",
                "What is the invoice number?",
                "What are the names of the two parties?"
            ]

            if run_on_cpu:
                model_name = 'bling-phi-3-gguf'
            else:
                server_uri_string = "http://11.123.456.789:8088"
                server_secret_key = "demo-test"
                ModelCatalog().setup_custom_llmware_inference_server(server_uri_string, secret_key=server_secret_key)
                model_name = "llmware-inference-server"

            self.prompter = Prompt().load_model(model_name)
            print("Initialization complete.")
            return True
        except Exception as e:
            print(f"Initialization failed: {e}")
            return False

    def process_invoices(self):
        if not self.prompter:
            print("Prompter is not initialized. Please initialize it first.")
            return

        print("Processing invoices...")
        invoices_path = os.path.join(Setup().load_sample_files(over_write=False), "Invoices")
        query_list = [
            "What is the total amount of the invoice?",
            "What is the invoice number?",
            "What are the names of the two parties?"
        ]

        total_invoices = sum(1 for _ in os.listdir(invoices_path) if _.endswith(".pdf"))
        processed_invoices = 0

        for i, invoice in enumerate(os.listdir(invoices_path)):
            if invoice.endswith(".pdf"):
                processed_invoices += 1
                progress = f"{processed_invoices}/{total_invoices}"
                print(f"\nProcessing invoice {progress}: {invoice}")

                for question in query_list:
                    source = self.prompter.add_source_document(invoices_path, invoice)
                    output = self.prompter.prompt_with_source(question, prompt_name="default_with_context")

                    for i, response in enumerate(output):
                        print("LLM Response -", question, "-", re.sub("[\n]", " ", response["llm_response"]))

                    self.prompter.clear_source_materials()

                print("Invoice processing complete.")

        print("All invoices processed.")
        print("Saving prompt state...")
        self.prompter.save_state()

        csv_output = HumanInTheLoop(self.prompter).export_current_interaction_to_csv()
        print("CSV output for human review:", csv_output)

        print("\nThank you for using our Invoice Processing System. Your feedback is valuable to us.")

    def run_with_user_interaction(self):
        print("Welcome to the Invoice Processing System!")
        while True:
            print("\nPlease choose an option:")
            print("1. Process invoices")
            print("2. Enter new invoices")
            print("3. Update existing invoices")
            print("4. View processed invoices")
            print("5. Quit")

            choice = input("Enter your choice: ")

            if choice == "1":
                if self.initialize_prompter():
                    self.process_invoices()
            elif choice == "2":
                self.enter_new_invoices()
            elif choice == "3":
                self.update_existing_invoices()
            elif choice == "4":
                self.view_processed_invoices()
            elif choice == "5":
                print("Thank you for using the Invoice Processing System. Goodbye!")
                break
            else:
                print("Invalid choice. Please choose again.")

    def enter_new_invoices(self):
        print("Enter details for the new invoice:")
        total_invoices = int(input("Enter the total number of invoices to enter: "))
        invoices_data = []
        for i in range(total_invoices):
            invoice_data = {}
            print(f"\nInvoice {i+1}:")
            invoice_data["total_amount"] = input("Enter the total amount of the invoice: ")
            invoice_data["invoice_number"] = input("Enter the invoice number: ")
            invoice_data["party_names"] = input("Enter the names of the two parties (separated by comma): ").split(",")
            invoices_data.append(invoice_data)
        print("\nNew invoices entered successfully.")

    def update_existing_invoices(self):
        print("Feature coming soon!")

    def view_processed_invoices(self):
        csv_file_path = input("Enter the path to the CSV file containing processed invoices: ")
        try:
            with open(csv_file_path, "r") as file:
                processed_data = file.read()
                print("Processed Invoices:")
                print(processed_data)
        except FileNotFoundError:
            print("File not found. Please enter a valid file path.")
        except Exception as e:
            print(f"An error occurred: {e}")

if __name__ == "__main__":
    processor = InvoiceProcessor()
    processor.run_with_user_interaction()


Welcome to the Invoice Processing System!

Please choose an option:
1. Process invoices on CPU
2. Process invoices on GPU/Server
3. Quit
Enter your choice: 1
Initializing...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

(…)n-phi-3-gguf_answer_sheet_042424_1.jsonl:   0%|          | 0.00/32.6k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/247k [00:00<?, ?B/s]

(…)-phi-3-gguf_core_rag_test_042424_1.jsonl:   0%|          | 0.00/66.6k [00:00<?, ?B/s]

bling-phi-3.gguf:   0%|          | 0.00/2.39G [00:00<?, ?B/s]

Initialization complete.
Processing invoices...

Processing invoice 1/40: zetq Simple Invoice .pdf
LLM Response - What is the total amount of the invoice? - Total: $330.00
LLM Response - What is the invoice number? - 23


KeyboardInterrupt: 